### Ingest qualifying json files

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [1]:
from pandas import read_csv,read_json,concat
from glob import glob
from datetime import datetime
from lib import configuration
from lib import common_functions

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [3]:
spark = common_functions.get_spark_session()

In [4]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [5]:
folder_path = f"{configuration.bronze_folder_path}/results/inc"
all_files = glob(folder_path + "/**/*.json", recursive = True)

dfs = [read_json(file, lines=True) for file in all_files]
results_df = concat(dfs, ignore_index=True)
results_df

,resultId,raceId_join,driverId_join,constructorId_join,number,grid,position,positionText,positionOrder,points,laps,statusId
0,202401,Bahrain Grand Prix2024,max_verstappen,red_bull,1,1,1,1,positionOrder,26,57,Finished
1,202401,Bahrain Grand Prix2024,perez,red_bull,11,5,2,2,positionOrder,18,57,Finished
2,202401,Bahrain Grand Prix2024,sainz,ferrari,55,4,3,3,positionOrder,15,57,Finished
3,202401,Bahrain Grand Prix2024,leclerc,ferrari,16,2,4,4,positionOrder,12,57,Finished
4,202401,Bahrain Grand Prix2024,russell,mercedes,63,3,5,5,positionOrder,10,57,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...
214,202411,Austrian Grand Prix2024,bottas,sauber,77,18,16,16,positionOrder,0,70,+1 Lap
215,202411,Austrian Grand Prix2024,zhou,sauber,24,20,17,17,positionOrder,0,70,+1 Lap
216,202411,Austrian Grand Prix2024,alonso,aston_martin,14,15,18,18,positionOrder,0,70,+1 Lap
217,202411,Austrian Grand Prix2024,sargeant,williams,2,19,19,19,positionOrder,0,69,+2 Laps


##### Step 2 - Rename columns and add new columns
1. Rename qualifyingId, driverId, constructorId and raceId
1. Add ingestion_date with current timestamp

In [6]:
results_df["data_source"] = configuration.v_data_source
#results_df["ingestion_date"] = common_functions.get_ingestion_date()
results_df

,resultId,raceId_join,driverId_join,constructorId_join,number,grid,position,positionText,positionOrder,points,laps,statusId,data_source
0,202401,Bahrain Grand Prix2024,max_verstappen,red_bull,1,1,1,1,positionOrder,26,57,Finished,api
1,202401,Bahrain Grand Prix2024,perez,red_bull,11,5,2,2,positionOrder,18,57,Finished,api
2,202401,Bahrain Grand Prix2024,sainz,ferrari,55,4,3,3,positionOrder,15,57,Finished,api
3,202401,Bahrain Grand Prix2024,leclerc,ferrari,16,2,4,4,positionOrder,12,57,Finished,api
4,202401,Bahrain Grand Prix2024,russell,mercedes,63,3,5,5,positionOrder,10,57,Finished,api
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,202411,Austrian Grand Prix2024,bottas,sauber,77,18,16,16,positionOrder,0,70,+1 Lap,api
215,202411,Austrian Grand Prix2024,zhou,sauber,24,20,17,17,positionOrder,0,70,+1 Lap,api
216,202411,Austrian Grand Prix2024,alonso,aston_martin,14,15,18,18,positionOrder,0,70,+1 Lap,api
217,202411,Austrian Grand Prix2024,sargeant,williams,2,19,19,19,positionOrder,0,69,+2 Laps,api


In [7]:
results_final_df = results_df.rename(columns={"resultId":"result_id","positionText":"position_text","positionOrder":"position_order","statusId":"status"})
results_final_df

,result_id,raceId_join,driverId_join,constructorId_join,number,grid,position,position_text,position_order,points,laps,status,data_source
0,202401,Bahrain Grand Prix2024,max_verstappen,red_bull,1,1,1,1,positionOrder,26,57,Finished,api
1,202401,Bahrain Grand Prix2024,perez,red_bull,11,5,2,2,positionOrder,18,57,Finished,api
2,202401,Bahrain Grand Prix2024,sainz,ferrari,55,4,3,3,positionOrder,15,57,Finished,api
3,202401,Bahrain Grand Prix2024,leclerc,ferrari,16,2,4,4,positionOrder,12,57,Finished,api
4,202401,Bahrain Grand Prix2024,russell,mercedes,63,3,5,5,positionOrder,10,57,Finished,api
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,202411,Austrian Grand Prix2024,bottas,sauber,77,18,16,16,positionOrder,0,70,+1 Lap,api
215,202411,Austrian Grand Prix2024,zhou,sauber,24,20,17,17,positionOrder,0,70,+1 Lap,api
216,202411,Austrian Grand Prix2024,alonso,aston_martin,14,15,18,18,positionOrder,0,70,+1 Lap,api
217,202411,Austrian Grand Prix2024,sargeant,williams,2,19,19,19,positionOrder,0,69,+2 Laps,api


In [8]:
drivers_df = read_csv(f'{configuration.silver_folder_path}/drivers.csv')
drivers_df = drivers_df[["driver_id","driver_ref"]]
drivers_df

,driver_id,driver_ref
0,1,abate
1,2,abecassis
2,3,acheson
3,4,adams
4,5,ader
...,...,...
854,855,zapico
855,856,zhou
856,857,zonta
857,858,zorzi


In [9]:
constructors_df = read_csv(f'{configuration.silver_folder_path}/constructors.csv')
constructors_df = constructors_df[["constructor_id","constructor_ref"]]
constructors_df

,constructor_id,constructor_ref
0,1,adams
1,2,afm
2,3,ags
3,4,alfa
4,5,alphatauri
...,...,...
207,208,watson
208,209,wetteroth
209,210,williams
210,211,wolf


In [10]:
race_circuits_df = read_csv(f'{configuration.silver_folder_path}/race_circuits.csv')
race_circuits_df = race_circuits_df[["race_id","race_id_join","race_date"]]
race_circuits_df

,race_id,race_id_join,race_date
0,81,German Grand Prix1959,1959-08-02
1,420,Australian Grand Prix1985,1985-11-03
2,436,Australian Grand Prix1986,1986-10-26
3,452,Australian Grand Prix1987,1987-11-15
4,468,Australian Grand Prix1988,1988-11-13
...,...,...,...
1120,319,Belgian Grand Prix1979,1979-05-13
1121,333,Belgian Grand Prix1980,1980-05-04
1122,347,Belgian Grand Prix1981,1981-05-17
1123,362,Belgian Grand Prix1982,1982-05-09


In [11]:
results_final_df = results_final_df.set_index("driverId_join").join(drivers_df.set_index("driver_ref"), lsuffix='driver_', how='inner').reset_index(drop=True)
results_final_df = results_final_df.set_index("constructorId_join").join(constructors_df.set_index("constructor_ref"), lsuffix='constructor_', how='inner').reset_index(drop=True)
results_final_df = results_final_df.set_index("raceId_join").join(race_circuits_df.set_index("race_id_join"), lsuffix='race_circuits_', how='inner').reset_index(drop=True)

results_final_df["ingestion_date"] = results_final_df["race_date"]

results_final_df = results_final_df[["result_id","race_id","driver_id","constructor_id","number","grid","position","position_text","position_order","points","laps","status","data_source","ingestion_date"]]
results_final_df

,result_id,race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,status,data_source,ingestion_date
0,202403,1104,305,6,10,17,13,13,positionOrder,0,57,+1 Lap,api,2024-03-24
1,202403,1104,573,6,31,15,16,16,positionOrder,0,57,+1 Lap,api,2024-03-24
2,202403,1104,17,14,14,10,8,8,positionOrder,4,58,Finished,api,2024-03-24
3,202403,1104,745,14,18,9,6,6,positionOrder,8,58,Finished,api,2024-03-24
4,202403,1104,456,75,16,4,2,2,positionOrder,19,58,Finished,api,2024-03-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,202410,1111,595,168,11,11,8,8,positionOrder,4,66,Finished,api,2024-06-23
215,202410,1111,98,171,77,12,16,16,positionOrder,0,65,+1 Lap,api,2024-06-23
216,202410,1111,856,171,24,15,13,13,positionOrder,0,65,+1 Lap,api,2024-06-23
217,202410,1111,11,210,23,0,18,18,positionOrder,0,65,+1 Lap,api,2024-06-23


##### Get max result_id already in silver and only append records newer than it.

In [12]:
max_result_id = common_functions.get_max_result_id()
max_result_id

The maximum value in the column 'result_id' is: 202410


202410

In [13]:
max_result_id = common_functions.get_max_result_id()
results_final_df = results_final_df[(results_final_df['result_id']>max_result_id)]
results_final_df

The maximum value in the column 'result_id' is: 202410


,result_id,race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,status,data_source,ingestion_date
19,202411,1112,305,6,10,13,10,10,positionOrder,1,71,Finished,api,2024-06-30
20,202411,1112,573,6,31,10,12,12,positionOrder,0,71,Finished,api,2024-06-30
21,202411,1112,17,14,14,15,18,18,positionOrder,0,70,+1 Lap,api,2024-06-30
22,202411,1112,745,14,18,17,13,13,positionOrder,0,70,+1 Lap,api,2024-06-30
23,202411,1112,456,75,16,6,11,11,positionOrder,0,71,Finished,api,2024-06-30
24,202411,1112,679,75,55,4,3,3,positionOrder,15,71,Finished,api,2024-06-30
25,202411,1112,385,85,27,9,6,6,positionOrder,8,71,Finished,api,2024-06-30
26,202411,1112,491,85,20,12,8,8,positionOrder,4,71,Finished,api,2024-06-30
27,202411,1112,569,135,4,2,20,20,positionOrder,0,64,+7 Laps,api,2024-06-30
28,202411,1112,604,135,81,7,2,2,positionOrder,18,71,Finished,api,2024-06-30


##### Write to output to processed container in parquet format

In [14]:
results_final_df.to_csv(f"{configuration.silver_folder_path}/results.csv", mode='a', header=False, index=False)

In [15]:
df_parquet = read_csv(f'{configuration.silver_folder_path}/results.csv')
df_parquet

,result_id,race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,status,data_source,ingestion_date
0,202005,1023,317,4,99.0,19,17,17,positionOrder,0.0,51,+1 Lap,api,2020-08-09
1,202005,1023,634,4,7.0,20,15,15,positionOrder,0.0,51,+1 Lap,api,2020-08-09
2,202005,1023,305,5,10.0,7,11,11,positionOrder,0.0,52,Finished,api,2020-08-09
3,202005,1023,436,5,26.0,16,10,10,positionOrder,1.0,52,Finished,api,2020-08-09
4,202005,1023,456,75,16.0,8,4,4,positionOrder,12.0,52,Finished,api,2020-08-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26494,202411,1112,595,168,11.0,8,7,7,positionOrder,6.0,71,Finished,api,2024-06-30
26495,202411,1112,98,171,77.0,18,16,16,positionOrder,0.0,70,+1 Lap,api,2024-06-30
26496,202411,1112,856,171,24.0,20,17,17,positionOrder,0.0,70,+1 Lap,api,2024-06-30
26497,202411,1112,11,210,23.0,16,15,15,positionOrder,0.0,70,+1 Lap,api,2024-06-30
